# <font color='darkblue'>Modul 5: Übungsmaterial zu Regression - Welches Auto verbraucht wie viel?</font> &nbsp; <font size='6'>&#128663;</font>

<hr style="border:1px solid gray"> </hr>

In diesem Notebook wird die Regression auf einen Datensatz über verschiedene Merkmale von Automodellen und deren Verbrauch angewendet. Es werden noch einmal die verschiedenen Arten der Regression durchgegangen und auch neue Methoden eingeführt. 

## <font color='darkblue'>Inhalt</font>


1. [Datenverständnis und Datenvorbereitung](#kap1)   
    

2. [Modellierung](#kap2)    
    2.1 [Train-Test-Split](#kap21)    
    2.2 [Merkmalsauswahl](#kap22)     
    2.3 [Normalisierung](#kap23)  
    2.4 [Eindimensionale Lineare Regression](#kap24)  
    2.5 [Eindimensionale Polynomiale Regression](#kap25)  
    2.6 [Mehrdimensionale Lineare Regression](#kap26)


3. [Regularisierung von Regressionsmodellen](#kap3)  
    3.1 [Ridge-Regression](#kap31)  


4. [Weitere Regressionsmodelle](#kap4)  
    4.1 [Support Vector Regression](#kap41)


5. [Fazit](#kap5)

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>1. Datenverständnis und Datenvorbereitung</font> <a name="kap1"></a>

Stellen Sie sich folgende Situation vor:
Sie sind bei einer Denkfabrik beschäftigt, die verschiedene Szenarien zur Senkung des Verbrauchs von Autos durchspielen soll. Eine Idee ist es, den Hubraum oder die Leistung der Autos oder beides zu begrenzen. Dazu wird ein Modell benötigt, das über alle Auotmodelle den Trend angibt - dies sind Regressionsfunktionen!

Das Ziel dieses Notebooks ist es also, eine möglichst gute Regressionsfunktion für eine Datensatz über Automodelle zu entwickeln. Dazu wird ein <a href="https://archive.ics.uci.edu/ml/datasets/auto+mpg">Autodatensatz</a> betrachtet. Er enthält Daten von 398 verschiedenen Modellen des amerikanischen Marktes aus dem Jahr 1993. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("M5_Uebung_Autos.csv")

Zunächst werden die Daten betrachtet, um einen groben Überblick über diese zu erhalten.

In [ ]:
df

Die Datenspalten sind in der folgenden Tabelle genauer beschrieben:

<table align='left'>
    <thead>
        <tr>
          <th style="text-align:left">Spaltenname</th>
          <th style="text-align:left">Bedeutung</th>
          <th style="text-align:left">Weitere Informationen</th>
        </tr>
    </thead>
    <tr>
    <td style="text-align:left; vertical-align: top">mpg</td>
    <td style="text-align:left; vertical-align: top">miles per gallon (dt. Meilen pro Gallone)</td>
    <td style="text-align:left; vertical-align: top">1 Meile = 1,609344 km; 1 Gallone = 3,7854 l</td>
    </tr>
    <tr>
    <td style="text-align:left">cylinders</td>
    <td style="text-align:left">Anzahl der Zylinder</td>
    <td style="text-align:left">-</td>
    </tr>
    <tr>
    <td style="text-align:left">displacement</td>
    <td style="text-align:left">Hubraum</td>
    <td style="text-align:left">[in³]</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">horsepower</td>
    <td style="text-align:left; vertical-align: top">Britische Pferdestärke (hp)</td>
    <td style="text-align:left; vertical-align: top">1 hp = 1,0139 PS </td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">weight</td>
    <td style="text-align:left; vertical-align: top">Gewicht</td>
    <td style="text-align:left; vertical-align: top">[lbs]</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">acceleration</td>
    <td style="text-align:left; vertical-align: top">Beschleunigung</td>
    <td style="text-align:left; vertical-align: top">Bei dieser Angabe handelt es sich um die Dauer [s], in der von 0 auf 60 mph beschleunigt wird. Das entspricht in etwa der in Deutschland üblichen Angabe der Zeitdauer für eine Beschleunigung von 0 auf 100 km/h.</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">model year</td>
    <td style="text-align:left; vertical-align: top">Baujahr</td>
    <td style="text-align:left; vertical-align: top">-</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">origin</td>
    <td style="text-align:left; vertical-align: top">Herkunftsland</td>
    <td style="text-align:left; vertical-align: top">1 = Amerika<br>2 = Europa<br>3 = Asien</td>
    </tr>
    <tr>
    <td style="text-align:left">car name</td>
    <td style="text-align:left">Bezeichnung des Autos</td>
    <td style="text-align:left">-</td>
    </tr>
</table>

<div class="alert alert-block alert-info">
&#128204; <b>Anmerkung:</b>  
    
Anders als in Deutschland üblich, wird der Verbrauch hier nicht in "Liter pro 100 km" angegeben, sondern in "Meilen pro Gallone". Es werden nicht nur andere Einheiten verwendet, sondern auch der "Kehrwert": Statt "Liter pro km" wären das also "km pro Liter" in metrischen Einheiten. Die Interpretation ist daher: Je höher der mpg-Wert desto weniger verbraucht es. Das stört aber nicht weiter, da die Angaben weiterhin recht intuitiv verständlich sind.

In [ ]:
df.info()

Es fällt auf, dass es keine NaN-Einträge gibt, d.h. alle Felder sind ordnungsgemäßt gefüllt. Allerdings ist die Spalte "horsepower" vom Typ "object", und nicht "float", dies wird weiter unten noch überarbeitet.

In [ ]:
df.hist(figsize=(10,10));

Die Daten sind in allen Merkmalen ungleichmäßig verteilt. Es gibt keine Ausreißer. Diese Merkmale brauchen nicht mehr angepasst zu werden.

Da ein allgemeines Modell für alle Autos entwickelt werden soll, werden hier die Merkmale Autoname und Herkunft nicht benötigt und sie werden gelöscht. Sollen die Verbräuche von Autofirmen oder nach dem Herkunftsland verglichen werden, müssten sie beibehalten werden. 

In [ ]:
df = df.drop('car name', axis=1)
df = df.drop('origin', axis=1)

Nun muss noch das Merkmal "horsepower" (Pferdestärke) untersucht werden. 

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 
    
Lassen Sie sich alle Ausprägungen ausgeben, die das Merkmal `horsepower` annimmt. 
    
Suchen Sie in der Liste nach Einträgen, die aus Zeichenketten (Strings) bestehen und überlegen Sie, wie Sie damit umgehen wollen.
    
Tipp: Methode `unique()`

In [ ]:
# Platz für Arbeitsauftrag

Die Ausgabe zeigt, dass "?" der einzige nicht-Zahl-Eintrag ist. Es werden nun alle Zeilen ausgegeben, bei denen "?" die Ausprägung des Merkmals ist:

In [ ]:
df[df["horsepower"] == "?"]

Nur bei diesen 6 Einträgen fehlt die PS-Angabe. 

Es gibt verschiedene Möglichkeiten, damit umzugehen:
- Die Einträge werden gelöscht
- Die Einträge werden durch die Mittelwerte der Spalte ersetzt
- Durch eine Recherche zu den betreffenden Automodellen werden echte Werte bestimmt

Da hier nur 6 Einträge betroffen sind, werden sie gelöscht. 

In [ ]:
df[df["horsepower"] == "?"].index

In [ ]:
df = df.drop(df[df["horsepower"] == "?"].index, axis=0)

Nun wird noch der Datentyp angepasst:

In [ ]:
df['horsepower'] = df['horsepower'].astype('float64')  

Mit den bereinigten Daten lohnt sich nun eine erste visuelle Begutachtung der Abhängigkeiten der Merkmale. 

In [ ]:
sns.pairplot(df);

Der pairplot gibt schon viele wichtige Erkenntnisse:
mpg wird von allen Merkmalen beeinflusst. Für die einzelnen Merkmale gilt:
- Je mehr Zylinder, desto weniger mpg
- Je größer das Gewicht, desto weniger mpg
- Je größer der Hubraum (displacement), desto weniger mpg
- Je stärker die Beschleunigung, desto größer wird mpg
- Je neuer, desto mehr mpg

Auch die starke Abhängigkeit der Merkmale untereinander ist zu erkennen. Beispielsweise nimmt bei steigendem Hubraum auch horsepower zu. Das ist nicht überraschend, kann aber für die Modellierung ungünstig sein. 

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>2. Modellierung</font> <a name="kap2"></a>

### <font color='darkblue'>2.1 Train-Test-Split</font> <a name="kap21"></a>

Nun sind die Daten soweit analysiert und vorbereitet, dass mit der Modellierung begonnen werden kann. Zunächst wird der Train-Test-Split durchgeführt, um die Testdaten bei allen weiteren Überlegungen außen vor zu lassen.

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Führen Sie einen Train-Test-Split durch, bei dem 70% der Daten der Trainingsmenge und 30% der Testmenge zugeordnet wird. Setzen Sie den `random_state` auf den Wert 1.  
</div>

In [ ]:
from sklearn.model_selection import train_test_split 
# train_set, test_set = train_test_split(df, )

### <font color='darkblue'>2.2 Merkmalsauswahl</font> <a name="kap22"></a>

Die Abhängigkeiten, die schon im Pairplot erkennbar waren, werden nun in der Korrelationsmatrix ausgedrückt:

In [ ]:
corr_matrix = train_set.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, vmin=-1, vmax=1)
plt.show()

Die Merkmale haben alle recht hohe Korrelationen mit dem Verbauch. Am größten ist sie bei dem Gewicht, gefolgt vom Hubraum, dann Pferdestärken und Anzahl der Zylinder. Beschleunigung und Modelljahr haben eine deutlich niedrigere Korrelation zu mpg.

Auffällig sind auch die hohen Korrelationen zwischen Zylinder, Hubraum, Pferdestärke und Gewicht. Diese Merkmale drücken in etwa das Gleiche aus. 

### <font color='darkblue'>2.3 Normalisierung</font> <a name="kap23"></a>

Für eine gewöhnliche Regression mittels einer Regressionsgeraden müssen die Daten normalerweise nicht normalisiert werden. Sollen aber weitere Regressionsmodelle ausprobiert werden, insbesondere welche mit Straftermen wie bei der schon bekannten Lasso-Regression, dann bietet es sich an, die Daten zu normalisieren. Dies wird nachfolgend vorgenommen: 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_set)                             # Skalierung wird an train_set angepasst
train_set_scaled = scaler.transform(train_set)    # Auf train_set anpassen
test_set_scaled = scaler.transform(test_set)      # Dieselbe Skalierung auch auf test_set anpassen
train_set_scaled = pd.DataFrame(train_set_scaled, index=train_set.index, columns=train_set.columns) # Rückumwandlung in Dataframe
test_set_scaled = pd.DataFrame(test_set_scaled, index=test_set.index, columns=test_set.columns)

In [ ]:
train_set_scaled.describe()

An den Einträgen für min und max ist erkennbar, dass nun alle Merkmale auf den Bereich von 0 bis 1 skaliert sind.

### <font color='darkblue'>2.4 Lineare Regression mit einem Merkmal</font> <a name="kap24"></a>

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 
    
Wählen Sie ein einzelnes Merkmal aus und bilden Sie ein lineares Regressionsmodell für das Zielmerkmal `mpg` in Abhängigkeit von dem gewählten Merkmal. Plotten Sie die Daten inklusive der Regressionsgeraden und lassen Sie sich die Gütemaße mse und $R^2$ sowohl für die Trainings- als auch für die Testdaten angeben.

Tipp: Video-Notebook

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Platz für Arbeitsauftrag

Sie sollten sehen: Die lineare Regression liefert schon ein recht gutes Ergebnis. 

### <font color='darkblue'>2.5 Eindimensionale Polynomiale Regression</font> <a name="kap25"></a>

Regression wird dazu benutzt, unbekannte Werte vorherzusagen, aber auch um generelle Abhängigkeiten genauer zu beschreiben. So ist es z.B. allgemein bekannt, das ein größeres Gewicht zu mehr Verbrauch führen wird. Aber wie stark wird der Verbrauch ansteigen? Ist das ein linearer Anstieg oder ist er quadratisch? 

Diese Frage kann eine polynomiale Regression zweiten Grades beantworten. 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly_features = PolynomialFeatures(degree=2, include_bias=False)  
X_polybig = poly_features.fit_transform(X_train)

quad_reg = LinearRegression()
quad_reg.fit(X_polybig, y_train)

In [ ]:
xline = np.linspace(-0.1,1.1, 100).reshape(100, 1) 
X_new_poly = poly_features.fit_transform(xline)
yline = quad_reg.predict(X_new_poly)
plt.plot(X_train, y_train, '.')
plt.plot(xline,yline , color='red')
plt.xlabel('weight')
plt.ylabel('mpg')
plt.xlim(-0.1, 1.1)
plt.show()
y_pred = quad_reg.predict(X_polybig)

print ('mse_training =', mean_squared_error(y_train, y_pred))
print ('r2_training =', r2_score(y_train, y_pred))
X_polybig_test = poly_features.fit_transform(X_test)
y_pred_test = quad_reg.predict(X_polybig_test)
print ('mse_test =', mean_squared_error(y_test, y_pred_test))
print ('r2_test =', r2_score(y_test, y_pred_test))

Tatsächlich sind die $R^2$-Werte hier besser.

Bei einer zu starken Extrapolation der Werte ist jedoch Vorsicht geboten: Da eine Parabel angepasst wurde, wird die Kurve rechts von der 1 irgendwo wieder ansteigen. Dies entspricht dann nicht mehr dem, was physikalisch sinnvoll ist!

### <font color='darkblue'>2.6 Mehrdimensionale Lineare Regression</font> <a name="kap26"></a>

Bei der Auswahl der Merkmale kann nicht nur die Korrelation jedes einzelnen Merkmals zum Zielmerkmal betrachtet werden, sondern auch die Korrelation untereinander, denn Merkmale, die untereinander hoch korreliert sind, drücken unter Umständen denselben Zusammenhang aus. Das Modell kann dann nichts Neues dazu lernen. Deswegen ist es manchmal sinnvoll, Merkmale auszuwählen, die nicht miteinander korreliert sind.

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Nachfolgend wird ein mehrdimensionales Regressionsmodell mit den Merkmalen `weight` und `model year` aufgestellt und seine Güte bestimmt. 
    
Stellen Sie analog ein Modell für die Merkmalen `weight` und `displacement` auf und vergleichen die Werte.

In [ ]:
X_train_m = train_set_scaled[["weight", "model year"]]
X_test_m = test_set_scaled[["weight", "model year"]]

m_reg = LinearRegression()
m_reg.fit(X_train_m, y_train)
print('Achsenabschnitt =', m_reg.intercept_)
print('Weitere_Koeffizienten =', m_reg.coef_)

y_pred_test = m_reg.predict(X_test_m)   

print ('mse_test= ', mean_squared_error(y_test, y_pred_test))
print ('r2_test= ', r2_score(y_test, y_pred_test))

In [ ]:
# Platz für Arbeitsauftrag

Es zeigt sich, dass das Modell für `weight`, `model year` wesentlich besser abschneidet. Dies liegt daran, dass das Modelljahr einen anderen Aspekt mit ins Modell einbringt, wogegen der Hubraum den gleichen Zusammenhang ausdrückt wie `weight`.

Nun wird das Modell auf alle Merkmale erweitert, um zu sehen, ob es dadurch besser wird:

In [ ]:
X_train_m = train_set_scaled.drop(["mpg"], axis=1)
X_test_m = test_set_scaled.drop(["mpg"], axis=1)

m_reg = LinearRegression()
m_reg.fit(X_train_m, y_train)
print('Achsenabschnitt =', m_reg.intercept_)
print('Weitere_Koeffizienten =', m_reg.coef_)

In [ ]:
y_pred_train = m_reg.predict(X_train_m)   

print ('mse_train= ', mean_squared_error(y_train, y_pred_train))
print ('r2_train= ', r2_score(y_train, y_pred_train))

y_pred_test = m_reg.predict(X_test_m)
print ('mse_test= ', mean_squared_error(y_test, y_pred_test))
print ('r2_test= ', r2_score(y_test, y_pred_test))

Das Modell mit allen Merkmalen ist nicht besser als das mit den Merkmalen `weight` und `model year`.

Bisher wurde nur die *lineare* Regression für mehrere Dimensionen angewendet. Es kann auch die polynomiale Regression angewendet werden. Dabei muss jedoch beachtet werden, dass dabei die Anzahl der Koeffizienten sehr stark wächst. Ein anderes Phänomen, das häufig auftritt, ist, dass die Koeffizienten immer größer werden. 

Wenn im folgenden Code der Grad der Polynome bei degree hochgesetzt wird, zeigt sich, dass auch hier die Koeffizienten stark ansteigen. 

In [ ]:
poly_features = PolynomialFeatures(degree=4, include_bias=False)  
X_polybig_m = poly_features.fit_transform(X_train_m)

quad_reg = LinearRegression()
quad_reg.fit(X_polybig_m, y_train)
print('Achsenabschnitt =', quad_reg.intercept_)
print('Weitere_Koeffizienten =', quad_reg.coef_)

y_pred = quad_reg.predict(X_polybig_m)

print ('mse_training =', mean_squared_error(y_train, y_pred))
print ('r2_training =', r2_score(y_train, y_pred))
X_polybig_test_m = poly_features.fit_transform(X_test_m)
y_pred_test = quad_reg.predict(X_polybig_test_m)
print ('mse_test =', mean_squared_error(y_test, y_pred_test))
print ('r2_test =', r2_score(y_test, y_pred_test))

Auch zu beobachten ist, dass das Trainingsmodell immer besser wird, aber die Ergebnisse für die Testdaten immer schlechter ($R^2$ ist negativ!). Das Modell lässt sich nicht mehr gut verallgemeinern. 

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>3. Regularisierung von Regressionsmodellen</font> <a name="kap3"></a>

Dieser Abschnitt beschäftigt sich mit der Frage, wie das Anwachsen der Koeffizienten und die zu starke Anpassung auf die Trainingsdaten (schlechte Verallgemeinerbarkeit) verhindert werden kann. "Regularisierung" heißt dabei, die Koeffizienten möglichst klein zu halten, ohne dabei allzu viel von der Güte eines Modells zu verlieren. 

### <font color='darkblue'>3.1 Ridge-Regression</font> <a name="kap31"></a>

Als erste Regularisierungsmethode haben Sie im Video schon die Lasso-Regression kennengelernt. Statt dem Mean Squared Error ($mse$) minimiert diese die Summe aus $mse$ und einem Extraterm:
$$mse + \alpha \sum_{i=1}^{m} |a_i|,$$
wobei $a_i$ die Regressionskoeffizienten sind.
Dadurch ergibt sich der Vorteil, dass die Lasso-Methode selbst schon eine Merkmalsauswahl trifft. Es können also ganz bequem erstmal alle Merkmale in das Modell gegeben werden.

Es gibt aber noch weitere Regularisierungsmethoden; eine davon ist die Ridge-Regression. Ähnlich wie bei der Lasso-Regression wird nicht der $mse$ minimiert, sondern ein zusätzlicher Term aufaddiert, hier 
$$mse + \alpha\sum_{i=1}^m |a_i|^2.$$ 
Der Unterschied besteht also "nur" in einem Quadrat, aber diese Methode verhält sich ganz anders. 

Ridge-Regression lohnt sich bei Datensätzen, die viele Merkmale haben, und deren Merkmale untereinander stark korreliert sind - so wie hier z.B. Hubraum mit Zylindern und Gewicht. Lasso-Regression oder auch eine manuelle Merkmalsauswahl würde eventuell nur eins der stark korrelierten Merkmale nutzen um die Komplexität des Modells zu verringern. Aber manchmal sollten alle Merkmale beachtet werden. Die "gewöhliche" lineare Regression würde dabei die Koeffizienten der korrelierten Merkmale überschätzen und das Modell etwas verzerren. Ridge-Regression vermeidet das und macht das Modell so stabiler. Es findet allerdings keine automatische Merkmalsauswahl statt. Diese sollte dann manuell vorher getroffen werden.



In [ ]:
X_train_m = train_set_scaled.drop(["mpg"], axis=1)
X_test_m = test_set_scaled.drop(["mpg"], axis=1)

In [ ]:
from sklearn.linear_model import Ridge

Die Syntax für die Ridge-Regression ist ganz ähnlich zur Lasso-Regression. Der Hyperparameter heißt auch hier $\alpha$.

In [ ]:
ridge_reg = Ridge(alpha=1)
ridge_reg.fit(X_train_m, y_train)

In [ ]:
print('Achsenabschnitt =', ridge_reg.intercept_)
print('Weitere_Koeffizienten =', ridge_reg.coef_)

In [ ]:
y_pred_train_ridge = ridge_reg.predict(X_train_m)   

print ('mse_train= ', mean_squared_error(y_train, y_pred_train_ridge))
print ('r2_train= ', r2_score(y_train, y_pred_train_ridge))

y_pred_test_ridge = ridge_reg.predict(X_test_m)
print ('mse_test= ', mean_squared_error(y_test, y_pred_test_ridge))
print ('r2_test= ', r2_score(y_test, y_pred_test_ridge))

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Probieren Sie die Ridge-Regression für verschiedene Werte von ``alpha`` aus. Gibt es Fälle, in denen eine Merkmalsauswahl getroffen wird, also manche Koeffizienten 0 sind? Vergleichen Sie zusätzlich die Koeffizienten mit den Werten der mehrdimensionalen Regression (mit allen Merkmalen). Sind die Koeffizienten hier größer oder kleiner oder lässt sich das nicht pauschal beantworten?

In [ ]:
# Platz für Arbeitsauftrag: 

Sie sollten sehen: Die Ridge-Regression für alle Merkmale erreicht hier ein ähnlich gutes Ergebnis wie bei der manuellen Merkmalsauswahl und hat gleichzeitig kleinere Koeffizienten.

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>4. Weitere Regressionsmodelle</font> <a name="kap4"></a>

Viele der Klassifikationsverfahren, die Sie in Modul 4 kennengelernt haben, können mit leichten Abwandlungen auch zur Regression verwendet werden. Sie heißen dann "Regressor" statt "Classificator". 

Hier soll das Vorgehen für die Support Vector Machines betrachtet werden, die im Falle der Regression als Support Vector Regression bezeichnet werden. 

### <font color='darkblue'>4.1 Support Vector Regression</font> <a name="kap41"></a>

Bei der Klassifikation wurde bei einer Support Vector Machine nach einer möglichst breiten Straße gesucht, die die beiden Klassen trennt. Nun wird diese Idee umgedreht:
Es wird nach einer Straße gesucht, die möglichst viele Punkte **enthält**. Dabei wird die Breite der Straße durch den Parameter `epsilon` vorgegeben. 

Zunächst wird der Support Vector Regressor auf nur ein Merkmal, nämlich `weight`, angewendet, um das Ergebnis anschaulich plotten zu können. Die Daten werden auf dieses Merkmal eingeschränkt und dann das Modell eingebunden, initialisiert und gefittet. Im Anschluss werden die Gütemaße ausgegeben und das Ergebnis geplottet. 

In [ ]:
X_train_m = train_set_scaled[["weight"]]
X_test_m = test_set_scaled[["weight"]]

y_train = y_train.values.ravel() # Befehl verändert die Form der Variable und verhindert so eine sonst auftretende Warnung 

from sklearn.svm import SVR
eps = 0.1  # Breite der Strasse
svm_reg = SVR(kernel='rbf', epsilon=eps)
svm_reg.fit(X_train_m, y_train)

In [ ]:
y_pred_train_svm = svm_reg.predict(X_train_m)   

print ('mse_train= ', mean_squared_error(y_train, y_pred_train_svm))
print ('r2_train= ', r2_score(y_train, y_pred_train_svm))

y_pred_test_svm = svm_reg.predict(X_test_m)
print ('mse_test= ', mean_squared_error(y_test, y_pred_test_svm))
print ('r2_test= ', r2_score(y_test, y_pred_test_svm))

In [ ]:
xline = np.linspace(-0.1,1.1, 100).reshape(100, 1) 
yline = svm_reg.predict(xline)
plt.plot(X_train, y_train, '.')
plt.plot(xline,yline , color='red')
plt.plot(xline,yline+eps , color='black')
plt.plot(xline,yline-eps , color='black')
plt.ylabel('mpg')
plt.xlim(-0.1, 1.1)
plt.title('Support Vector Regression mit Straße')
plt.show()

Die Anpassung ist in etwa so gut wie bei der linearen Regression. Die Kurve kann aber - abhängig von den Daten - beliebige Formen annehmen. 

Nun soll der Support Vector Regressor auf alle Merkmale angewendet werden, um zu sehen, ob so bessere Ergebnisse erzeugt werden. Hierfür werden die Mengen wieder neu definiert: 

In [ ]:
X_train_m = train_set_scaled.drop(["mpg"], axis=1)
X_test_m = test_set_scaled.drop(["mpg"], axis=1)

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Wenden Sie dasselbe Modell wie oben nun auf die neu definierten Daten an. 

In [ ]:
# Hier soll wieder ein SVR gefittet werden.

Die Ausgabe der Gütemaße zeigt: Die SVR mit allen Merkmalen liefert anhand der Gütemaße das bisher beste Modell:

In [ ]:
y_pred_train_svm = svm_reg.predict(X_train_m)   

print ('mse_train= ', mean_squared_error(y_train, y_pred_train_svm))
print ('r2_train= ', r2_score(y_train, y_pred_train_svm))

y_pred_test_svm = svm_reg.predict(X_test_m)
print ('mse_test= ', mean_squared_error(y_test, y_pred_test_svm))
print ('r2_test= ', r2_score(y_test, y_pred_test_svm))

Eine weitere Verbesserung könnte durch die Anpassung der Hyperparameter `epsilon` und `C` erreicht werden. 

Allgemein erzeugt die Support Vector Regression gute Regressionsfunktionen. Sie können vielfältigere Formen annehmen als die vorgebenen linearen oder polynomialen Regressionsfunktionen. 

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>5. Fazit</font> <a name="kap5"></a>

Dieses Notebook hat:
- die Schritte der Datenvorbereitung und Datenbereinigung wiederholt.
- die Schritte der Modellierung wiederholt.
- Ihre Fähigkeiten geschult, eine eindimensionale lineare Regression, eine eindimensionale polynomiale Regression sowie eine mehrdimensionale lineare Regression durchzuführen.
- ein neues Verfahren, nämlich die Ridge-Regression, eingeführt.
- die Übertragung der Support Vector Machine auf die Regression, also die Support Vector Regression, dargestellt.